### 3. GenAi-task

In [33]:
import pandas as pd
import os
import random
from dotenv import load_dotenv

from huggingface_hub import login

In [34]:
# Load environment variables from the .env file
load_dotenv()

# Access the Hugging Face token
hf_token = os.getenv("HF_TOKEN")

# Use the token to authenticate with Hugging Face
login(token=hf_token)

In [35]:
project_path = 'dataset/interim/cleaned_with_sentiment_numeric.csv'
categories_path = 'dataset/interim/amazon_review_categories.csv'

In [36]:
df = pd.read_csv(project_path)

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_11138/905003903.py:1: DtypeWarning: Columns (1,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(project_path)


In [37]:
df_categories = pd.read_csv(categories_path)

In [38]:
df_categories.head()

,category,dominant_topic,label
0,"Fire Tablets,Tablets,Computers & Tablets,All T...",2,tablets
1,"Stereos,Remote Controls,Amazon Echo,Audio Dock...",3,home
2,"Back To College,College Electronics,College Tv...",4,Topic 4
3,"Walmart for Business,Office Electronics,Tablet...",2,tablets
4,"Electronics,iPad & Tablets,All Tablets,Fire Ta...",2,tablets


In [39]:
#clean names:
# Clean product names: remove special characters, extra commas, and strip whitespace
df['name'] = df['name'].replace({r'\r': ' ', r'\n': ' ', ',,': ','}, regex=True).str.strip()

# Remove any remaining multiple spaces
df['name'] = df['name'].replace(r'\s+', ' ', regex=True)

In [40]:
df['name_clean'] = df['name'].str.split(',').str[0]

In [41]:
df_all = pd.merge(df,df_categories[['category','label']],how='left',left_on='categories',right_on='category')

In [42]:
grouped_reviews = df_all.groupby(['label','name_clean'])['reviews.text'].apply(list).reset_index()

In [43]:
# Assume the column with lists is named 'list_column'
# Create a new column 'list_length' that stores the number of items in each list
grouped_reviews['nr_reviews'] = grouped_reviews['reviews.text'].apply(len)

In [44]:
ratings_helper = pd.DataFrame(df_all.groupby('name_clean')['sentiment'].mean()).reset_index()

In [45]:
df_final = pd.merge(grouped_reviews,ratings_helper,on='name_clean')

In [46]:
df_final_filtered = df_final[df_final['nr_reviews']>=5]

In [47]:
df_final_filtered.head(2)

,label,name_clean,reviews.text,nr_reviews,sentiment
0,Topic 4,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,"[Having received my Fire TV today, I've been t...",6,1.920039
1,Topic 4,Echo (White),[Stay away from the certified refurbished Amaz...,8,1.941088


In [48]:
df_sorted = df_final_filtered.sort_values(by=['label', 'sentiment'], ascending=[True, False])

# Group by 'category' and select the top 3 rows per group
top_3_per_category = df_sorted.groupby('label').head(3)
last_3_per_category = df_sorted.groupby('label').tail(3)

In [49]:
top_3_per_category['cat'] = '5 Star'
last_3_per_category['cat'] = 'Fiasko'

top_categories = pd.concat([top_3_per_category,last_3_per_category])
top_categories = top_categories.drop_duplicates(subset=['name_clean'])

/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_11138/1094808961.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  top_3_per_category['cat'] = '5 Star'
/var/folders/qk/10jspwn51hb5gk7bfj_xq96h0000gn/T/ipykernel_11138/1094808961.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  last_3_per_category['cat'] = 'Fiasko'


In [50]:
import random

# Function to randomly select 9 reviews from the list of reviews
def select_random_reviews(reviews_list):
    # If there are fewer than 3 reviews, return them all
    if len(reviews_list) <= 20:
        return reviews_list
    # Otherwise, return 3 random reviews
    return random.sample(reviews_list, 20)

In [51]:
top_categories['reviews.text'] = top_categories['reviews.text'].apply(select_random_reviews)

In [52]:
for f in top_categories['reviews.text']:
    print(f)

['Stay away from the certified refurbished Amazon Fire TV. I received two Certified Refurbished defective fire TVs. The first the screen looked half corrupted, see the pictures. The second Certified Refurbished Amazon fire tv I received would not boot up, it was stuck in an infinite loop where I would only see the amazon logo for 10 seconds then it would boot up again.If anything, Amazon needs to re-visit its refurbishment process and quality control of the refurbished products they let get in consumer hands.Update 11/28/16: I purchased a new Fire TV during the BF sale and I have had no issues so far :).', "I seriously wonder why it took me to get FireTV for so long. I originally had a Roku XS and then I stupidly bought a Roku 4. Both devices were all right but neither could handle 5.1 audio from Netflix on my Vizio sound bar (essentially your device must be cable of handling Dolby Digital Plus and nothing less using Roku). I finally decided to give FireTV a try and I'm amazed. Not onl

## GenAi


#### SUMMARIZATION

In [53]:

# Function to randomly select  reviews and generate a summary
def summarize_reviews(reviews_list, num_reviews=15):
    # If there are fewer reviews than num_reviews, use all reviews
    if len(reviews_list) < num_reviews:
        selected_reviews = reviews_list
    else:
        # Randomly select 5 reviews from the list
        selected_reviews = random.sample(reviews_list, num_reviews)
    
    # Convert the selected reviews to a single string
    reviews_text = " ".join(selected_reviews)
    
    # Generate a summary
    summary = summarizer(reviews_text, max_length=150, min_length=50, do_sample=False)
    
    # Return the summary text
    return summary[0]['summary_text']

In [54]:
import torch
from transformers import pipeline
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")

# Load the summarization pipeline (you can use BART or another model)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=0 if torch.backends.mps.is_available() else -1)

In [55]:
top_categories['summary'] = top_categories['reviews.text'].apply(summarize_reviews)

    # Save the DataFrame to a new CSV
top_categories.to_csv('dataset/interim/products_with_summaries.csv', index=False)

    # Display the updated DataFrame with summaries

Your max_length is set to 150, but your input_length is only 118. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=59)


In [56]:
top_categories.head()

,label,name_clean,reviews.text,nr_reviews,sentiment,cat,summary
1,Topic 4,Echo (White),[Stay away from the certified refurbished Amaz...,8,1.941088,5 Star,This is one of the best purchases I have made ...
0,Topic 4,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,"[Having received my Fire TV today, I've been t...",6,1.920039,5 Star,The image quality (movies and games) is excell...
6,home,Amazon Fire Hd 6 Standing Protective Case(4th ...,[My family really enjoys the Amazon Echo. We h...,7,2.000000,5 Star,The Amazon Echo is a great way to control your...
13,home,Kindle Dx Leather Cover,[Purchased the Echo as a bridal shower gift. M...,9,2.000000,5 Star,The Amazon Echo has very good voice recognitio...
4,home,Amazon 9W PowerFast Official OEM USB Charger a...,[I have been using this for awhile and it defi...,36,1.972222,5 Star,Alexa is like having music on demand and so mu...


### Cerate Posts:

# Generate blogposts blogposts:

In [57]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load the tokenizer and model
model_name = "EleutherAI/gpt-neo-1.3B" 
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Set the device to GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move model to the correct device

# Initialize the pipeline for text generation
generator = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0 if device.type == "cuda" else -1)

def generate_blog_post(product_name, summary):
    prompt = f"Write a blog post about the product '{product_name}' based on the following review summary: {summary}"
    
    inputs = tokenizer(prompt, return_tensors="pt").to(device)  # Move inputs to the correct device
    input_ids = inputs["input_ids"]
    
    # Generate text with attention_mask and max_length control
    outputs = model.generate(
        input_ids, 
        attention_mask=inputs["attention_mask"],  # Add attention_mask to inputs
        max_length=500, 
        num_return_sequences=1, 
        no_repeat_ngram_size=2, 
        temperature=0.7,
        pad_token_id=tokenizer.eos_token_id  # Set pad_token_id explicitly
    )

    # Decode the generated text
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Split the text at the first two new lines (\n\n) and keep the part after the split
    blog_post = generated_text.split("\n\n", 1)[-1].strip()  # Ensure you only keep the blog post text

    return blog_post

In [58]:
# Add a new column to store the generated blog posts
top_categories['blogpost'] = ''

# Loop over each row in the DataFrame and generate the blog post
for idx, row in top_categories.iterrows():
    
    product_name = row['name_clean']
    summary = row['summary']
    cat = row['cat']
    print(product_name)
    print(cat)
    print(summary)
    # Generate the blog post for the current product
    blog_post = generate_blog_post(product_name, summary)
    
    # Store the generated blog post in the new 'blogpost' column for the current row
    top_categories.at[idx, 'blogpost'] = blog_post

# Print a sample blog post for a category
for idx, row in top_categories.iterrows():
    print(f"Blog post for product '{row['name_clean']}' in category '{row['label']}':\n{row['blogpost']}\n")

Echo (White)
5 Star
This is one of the best purchases I have made in the past year. Love it, had the stick and it was kinda slow navigating. This one is much faster. Was going to use this and cancel cable. When I told my cable company I wanted to cancel and I was just going to stream everything they cut my bundle cost by 100.


/Users/weemoe/Documents/Projects/ironhack_project/project_amazon/venv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue
5 Star
The image quality (movies and games) is excellent. Navigation is very straightforward and it's generally easy to find anything you're looking for. Installation of games and apps goes very smoothly and generally pretty quickly. As far as I can tell, there's no way to limit the list of movies and tv shows to what is available for Amazon Prime members.
Amazon Fire Hd 6 Standing Protective Case(4th Generation - 2014 Release)
5 Star
The Amazon Echo is a great way to control your TV and music. The sound is good, and it gets really loud. Set up was easy. New features are always added. It is compatible with a growing list of products. This will again revolutionize home functionality.
Kindle Dx Leather Cover
5 Star
The Amazon Echo has very good voice recognition. It works just as advertised and is quick with the responses. The setup was easy and quick. Next purchase...smart plugs so I can turn some items off and on wi

In [59]:
for idx, row in top_categories.iterrows():
    print(f"Blog post for product '{row['name_clean']}' in category '{row['label']}':\n{row['blogpost']}\n")

Blog post for product 'Echo (White)' in category 'Topic 4':
I have been using this for a few months now and have had no problems. The only thing I would change is to change the color of it. It is a little too dark for me. But I love it!
The only problem I had was with the remote. My remote was broken and the only way to get it to work was to buy a new one. So I bought a replacement remote and that worked great. Now I can use the Echo (white) with my Echo Dot. And I don't have to worry about my remote breaking. Great product! I will be buying more! :)
Thanks for the great product. :) I really like it and will continue to purchase. Thanks!

Blog post for product 'Brand New Amazon Kindle Fire 16gb 7 Ips Display Tablet Wifi 16 Gb Blue' in category 'Topic 4':
The Kindle has a lot of potential, especially if it can be made to work with Amazon's Prime service. But it doesn't have the features that Amazon needs to make it a successful product. Amazon has to do a better job of making the Kindle

In [60]:
top_categories

,label,name_clean,reviews.text,nr_reviews,sentiment,cat,summary,blogpost
1,Topic 4,Echo (White),[Stay away from the certified refurbished Amaz...,8,1.941088,5 Star,This is one of the best purchases I have made ...,I have been using this for a few months now an...
0,Topic 4,Brand New Amazon Kindle Fire 16gb 7 Ips Displa...,"[Having received my Fire TV today, I've been t...",6,1.920039,5 Star,The image quality (movies and games) is excell...,"The Kindle has a lot of potential, especially ..."
6,home,Amazon Fire Hd 6 Standing Protective Case(4th ...,[My family really enjoys the Amazon Echo. We h...,7,2.000000,5 Star,The Amazon Echo is a great way to control your...,The Amazon Fire HD 6 is the latest addition to...
13,home,Kindle Dx Leather Cover,[Purchased the Echo as a bridal shower gift. M...,9,2.000000,5 Star,The Amazon Echo has very good voice recognitio...,I have a Kindle Fire HD 8.9 and I love it. I h...
4,home,Amazon 9W PowerFast Official OEM USB Charger a...,[I have been using this for awhile and it defi...,36,1.972222,5 Star,Alexa is like having music on demand and so mu...,I bought this charger for the Amazon Echo and ...
17,tablets,Amazon - Fire 16GB (5th Gen,"[I love my Kindle Voyage, however it does not ...",5,2.000000,5 Star,"I love my Kindle Voyage, however it does not h...",I have been using the Amazon Fire HD for a cou...
20,tablets,Amazon Echo and Fire TV Power Adapter,[This is the first e-reader I've gotten and I ...,8,2.000000,5 Star,This is the first e-reader I've gotten and I L...,I've been using the Echo for a few weeks now a...
22,tablets,Amazon Fire Kids Edition Tablet,[The text is sharp and paper like in well lit ...,6,2.000000,5 Star,Slim and lightweight when not using a case whi...,The tablet is very light and easy to hold. It ...
12,home,Fire Tablet,[I initially had trouble deciding between the ...,19,1.908185,Fiasko,Kindle Paperwhite launched July 2015. It has a...,The Fire Tablet is a tablet that is designed t...
14,home,New Amazon Kindle Fire Hd 9w Powerfast Adapter...,[Just love it. Easy to set up and use. Tons of...,7,1.857143,Fiasko,Alexa is my new favorite device. Easy to set u...,I have a Kindle and a Fire. The Kindle is grea...


#### export results

In [62]:
import sqlite3

import sqlite3

# Connect to the SQLite database
db_path = "flask_project/data/blogposts.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Drop the 'blogposts' table if it exists
cursor.execute('''
DROP TABLE IF EXISTS blogposts
''')

# Create the 'blogposts' table with columns based on the DataFrame, including 'id' as the primary key
cursor.execute('''
CREATE TABLE blogposts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    label TEXT,
    name_clean TEXT,
    reviews_text TEXT,
    nr_reviews INTEGER,
    sentiment REAL,
    cat TEXT,
    summary TEXT,
    blogpost TEXT
)
''')

# Commit the table creation
conn.commit()

# Insert data from the DataFrame into the 'blogposts' table
for idx, row in top_categories.iterrows():
    cursor.execute('''
        INSERT INTO blogposts (label, name_clean, reviews_text, nr_reviews, sentiment, cat, summary, blogpost)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (row['label'], row['name_clean'], ', '.join(row['reviews.text']), row['nr_reviews'], row['sentiment'], row['cat'], row['summary'], row['blogpost']))

# Commit the inserts
conn.commit()

# Close the connection
conn.close()

print("Data successfully saved to the 'blogposts' table with 'id' in blogposts.db.")

Data successfully saved to the 'blogposts' table with 'id' in blogposts.db.
